In [103]:
import tensorflow as tf
import numpy as np
import data

In [104]:
vocab_size = 1681
d, id_to_token = data.get_data(vocab_size)
for k in id_to_token:
    if id_to_token[k] == 'eos':
        id_to_token[k] = '\n'
print(len(d))
max_word_length = max([len(id_to_token[np.argmax(j)]) for j in d])
print(max_word_length)
print(id_to_token[22])

['i', 'would', 'like', 'to', 'talk', 'today', 'about', 'how', 'to', 'develop', 'a', 'new', 'foreign', 'policy', 'direction', 'for', 'our', 'country', 'one', 'that', 'replaces', 'randomness', 'with', 'purpose', 'ideology', 'with', 'strategy', 'and', 'chaos', 'with', 'peace', 'it', 'is', 'time', 'to', 'shake', 'the', 'rust', 'off', 'of']
1681
10229
15
re


In [105]:
c_to_i, embedding = data.get_char_embedding()

def word_generator(ix, num=10):
    return d[ix : ix + num]

def sequence_char_matrix(data, ix, num = 10):
    words = [id_to_token[np.argmax(j)] for j in data[ix : ix + num]]
    chars = [embedding[[c_to_i[c] for c in w]] for w in words]
    to_pad = [max_word_length - t for t in map(lambda x: len(x), words)]
    pad = [np.zeros([tp, embedding.shape[0]]) for tp in to_pad]
    reshape = [1, max_word_length, embedding.shape[0], 1]
    padded = np.concatenate([np.reshape(np.r_[ch, pd], reshape) for ch, pd in zip(chars, pad)], axis=0)
    return padded

a = sequence_char_matrix([d[0], d[0]], 0, 2)
_ = word_generator(0, 13)

In [106]:
# CNN hyperparameters
input_height = max_word_length
input_width = embedding.shape[0]
batch_size = 13
print(input_height, input_width)
filter_heights = [2, 3, 4]
feature_maps = [3, 4, 5]

15 87


In [107]:
def inference(inp):
    initializer = tf.random_uniform_initializer(minval=-0.1, maxval=0.1, dtype=tf.float32)
    with tf.variable_scope('conv1') as scope:        
        weight = tf.get_variable('weights', [filter_heights[0], input_width, 1, feature_maps[0]], initializer=initializer)
        conv1 = tf.nn.tanh(tf.nn.conv2d(inp, weight, strides=[1, 1, 1, 1], padding='VALID'))
    pool1 = tf.squeeze(tf.nn.max_pool(conv1, ksize=[1, conv1.get_shape()[1], 1, 1], strides=[1, 1, 1, 1], padding='VALID'), squeeze_dims=[1, 2])
    
    with tf.variable_scope('conv2') as scope:        
        weight = tf.get_variable('weights', [filter_heights[1], input_width, 1, feature_maps[1]], initializer=initializer)
        conv2 = tf.nn.tanh(tf.nn.conv2d(inp, weight, strides=[1, 1, 1, 1], padding='VALID'))
    pool2 = tf.squeeze(tf.nn.max_pool(conv2, ksize=[1, conv2.get_shape()[1], 1, 1], strides=[1, 1, 1, 1], padding='VALID'), squeeze_dims=[1, 2])
    
    with tf.variable_scope('conv3') as scope:        
        weight = tf.get_variable('weights', [filter_heights[2], input_width, 1, feature_maps[2]], initializer=initializer)
        conv3 = tf.nn.tanh(tf.nn.conv2d(inp, weight, strides=[1, 1, 1, 1], padding='VALID'))
    pool3 = tf.squeeze(tf.nn.max_pool(conv3, ksize=[1, conv3.get_shape()[1], 1, 1], strides=[1, 1, 1, 1], padding='VALID'), squeeze_dims=[1, 2])
    pool_total = tf.concat(1, [pool1, pool2, pool3], name='total_pool')
    return pool_total

In [108]:
tf.reset_default_graph()
input_ = tf.placeholder(shape=[None, max_word_length, embedding.shape[0], 1], dtype=tf.float32, name = 'cnn_in')
with tf.variable_scope("CNN") as scope:
    cnn_output = inference(input_)

In [109]:
# highway network
initializer = tf.random_uniform_initializer(minval=-0.1, maxval=0.1, dtype=tf.float32)
# initializer = tf.random_normal_initializer(mean=0, stddev=0.001, dtype=tf.float32)
# transformation gate parameters
W_t = tf.get_variable('Wt', shape=[sum(feature_maps)], initializer=initializer)
b_t = tf.get_variable('bt', shape=[sum(feature_maps)], initializer=initializer)
# highway parameters
W_h = tf.get_variable('Wh', shape=[sum(feature_maps)], initializer=initializer)
b_h = tf.get_variable('bh', shape=[sum(feature_maps)], initializer=initializer)
# one layer highway network
t = tf.nn.sigmoid(W_t * cnn_output + b_t)
highway_output = t * tf.nn.tanh(W_h * cnn_output + b_h) + (tf.ones_like(t) -  t) * cnn_output

In [124]:
# RNN hyperparameters

epochs = 5000
hidden_layer = 128
input_size = sum(feature_maps)
output_size = vocab_size
learning_rate = 0.1
print_step = 1000
num_steps = batch_size
input_size, output_size

(12, 1681)

In [111]:
# initializer = tf.random_uniform_initializer(minval=-0.1, maxval=0.1, dtype=tf.float32)
initializer = tf.random_uniform_initializer(minval=-0.1, maxval=0.1, dtype=tf.float32)
Wxh = tf.get_variable('Wxh', shape=[input_size, hidden_layer], initializer=initializer)
Whh = tf.get_variable('Whh', shape=[hidden_layer, hidden_layer], initializer=initializer)
Why = tf.get_variable('Why',shape=[hidden_layer, output_size], initializer=initializer)
by = tf.get_variable('by', shape=[output_size], initializer=initializer)
# weights associated with update gate
Wxz = tf.get_variable('Wxz', shape=[input_size, hidden_layer], initializer=initializer)
Whz = tf.get_variable('Whz', shape=[hidden_layer, hidden_layer], initializer=initializer)
# weights associated with the reset gate
Wxr = tf.get_variable('Wxr', shape=[input_size, hidden_layer], initializer=initializer)
Whr = tf.get_variable('Whr', shape=[hidden_layer, hidden_layer], initializer=initializer)

In [112]:
def GRU(prev, inp):
    i = tf.reshape(inp, shape=[1, -1])
    p = tf.reshape(prev, shape=[1, -1])
    z = tf.nn.sigmoid(tf.matmul(i, Wxz) + tf.matmul(p, Whz))    # update gate
    r = tf.nn.sigmoid(tf.matmul(i, Wxr) + tf.matmul(p, Whr))    # reset gate
    h_ = tf.nn.tanh(tf.matmul(i, Wxh) + tf.matmul(tf.mul(p, r), Whh))
    h = tf.mul(tf.sub(tf.ones_like(z), z), h_) + tf.mul(z, p)
    return tf.reshape(h, [hidden_layer])

In [113]:
b = tf.placeholder(shape=[None, vocab_size], dtype=tf.float32, name='targets')
initial = tf.placeholder(shape=[hidden_layer], dtype=tf.float32)
states = tf.scan(GRU, highway_output, initializer=initial)
outputs = tf.nn.softmax(tf.matmul(states, Why) + by, name='model_out')
# loss = -tf.reduce_mean(b * tf.log(outputs))
loss =  - tf.reduce_mean(tf.reduce_sum(b * tf.log(outputs), 1))
optimizer = tf.train.AdagradOptimizer(learning_rate)

# clipping gradients between -1 and 1.
grad_var_pairs = optimizer.compute_gradients(loss, tf.trainable_variables())
clipped_grad_var_pairs = [(tf.clip_by_value(gv[0], -1, 1), gv[1]) for gv in grad_var_pairs]
optimize_op = optimizer.apply_gradients(clipped_grad_var_pairs)

In [114]:
def generate(ix, num_steps):    
    seed = word_generator(ix, 1)
    text = [np.argmax(seed[0])]
    h = np.zeros(hidden_layer)
    for i in range(num_steps):
        feed = {input_: np.array(sequence_char_matrix(seed, 0, 1)), initial: h}
        out, e = sess.run([outputs, states], feed_dict=feed)
        text.append(np.argmax(out[0]))
        seed = out
        h = e[-1]
    return [id_to_token[j] for j in text]

In [115]:
sess = tf.Session()
ix = 0
sess.run(tf.initialize_all_variables())
losses = []
h = np.zeros(hidden_layer)

In [125]:
for i in range(epochs):
    if ix + num_steps >= len(d):
        ix = 0
    x = np.array(sequence_char_matrix(d, ix, batch_size))
    y = np.array(word_generator(ix + 1, batch_size))
    feed = {input_: x, initial: np.zeros(hidden_layer), b: y}    
    o, l, _ = sess.run([outputs, loss, optimize_op], feed_dict=feed)
    ix += num_steps
    losses.append(l)
    if i % print_step == 0:
        print('Epoch', i)
        print(l)

Epoch 0
3.07657
Epoch 1000
3.29876
Epoch 2000
2.51236
Epoch 3000
2.30238
Epoch 4000
1.96116


In [126]:
print(generate(10, 70))



['a', 'year', 'that', 's', 'going', 'to', 'have', 'people', 'that', 's', 'the', 'disaster', 'that', 's', 'coming', 'nice', 'in', 'the', 'other', 'way', 'the', 'bridge', 'trade', 'off', 'our', 'country', 'we', 've', 'been', 'allowed', 'that', 'aren', 't', 'close', 'him', 'from', 'who', 'is', 'happening', 'and', 'now', 'we', 'have', 'to', 'do', 'you', 'know', 'we', 'have', 'a', 'political', 'political', 'he', 'said', 'it', 's', 'a', 'nation', 'of', 'it', 'said', 'me', 'wanted', 'but', 'we', 'will', 'be', 'accused', 'i', 'said', 'it']
